In [1]:
# Import libs

import psycopg2
import pandas as pd
import numpy as np

In [2]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname = postgres user = postgres password = torontoraptors")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create Sparkify database with UTF8 encoding 
    cur.execute("DROP DATABASE IF EXISTS countries")
    cur.execute("CREATE DATABASE countries")

    #Close connection to default database 
    conn.close()

    #Connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname = countries user = postgres password = torontoraptors")
    cur = conn.cursor()

    return cur, conn



In [3]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
#Create Dataframe
country_data = pd.read_csv("country_profile_variables.csv")

In [6]:
country_data.head()

,country,Region,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),...,Mobile-cellular subscriptions (per 100 inhabitants).1,Individuals using the Internet (per 100 inhabitants),Threatened species (number),Forested area (% of land area),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Energy supply per capita (Gigajoules),"Pop. using improved drinking water (urban/rural, %)","Pop. using improved sanitation facilities (urban/rural, %)",Net Official Development Assist. received (% of GNI)
0,Afghanistan,SouthernAsia,652864,35530,54.4,106.3,20270,-2.4,623.2,23.3,...,8.3,42,2.1,9.8/0.3,63,5,78.2/47.0,45.1/27.0,21.43,-99
1,Albania,SouthernEurope,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,...,63.3,130,28.2,5.7/2.0,84,36,94.9/95.2,95.5/90.2,2.96,-99
2,Algeria,NorthernAfrica,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,...,38.2,135,0.8,145.4/3.7,5900,55,84.3/81.8,89.8/82.2,0.05,-99
3,American Samoa,Polynesia,199,56,278.2,103.6,-99,-99,-99.0,-99,...,-99,92,87.9,-99,-99,-99,100.0/100.0,62.5/62.5,-99,-99
4,Andorra,SouthernEurope,468,77,163.8,102.3,2812,0.8,39896.4,0.5,...,96.9,13,34.0,0.5/6.4,1,119,100.0/100.0,100.0/100.0,-99,-99


In [7]:
#List all columns
column_headers = list(country_data.columns.values)
print(column_headers)

['country', 'Region', 'Surface area (km2)', 'Population in thousands (2017)', 'Population density (per km2, 2017)', 'Sex ratio (m per 100 f, 2017)', 'GDP: Gross domestic product (million current US$)', 'GDP growth rate (annual %, const. 2005 prices)', 'GDP per capita (current US$)', 'Economy: Agriculture (% of GVA)', 'Economy: Industry (% of GVA)', 'Economy: Services and other activity (% of GVA)', 'Employment: Agriculture (% of employed)', 'Employment: Industry (% of employed)', 'Employment: Services (% of employed)', 'Unemployment (% of labour force)', 'Labour force participation (female/male pop. %)', 'Agricultural production index (2004-2006=100)', 'Food production index (2004-2006=100)', 'International trade: Exports (million US$)', 'International trade: Imports (million US$)', 'International trade: Balance (million US$)', 'Balance of payments, current account (million US$)', 'Population growth rate (average annual %)', 'Urban population (% of total population)', 'Urban population

In [8]:
#Select certain columns you wish to keep 

country_data_clean = country_data[['country', 'GDP: Gross domestic product (million current US$)', 'GDP growth rate (annual %, const. 2005 prices)', 'GDP per capita (current US$)', 'Economy: Agriculture (% of GVA)', 'Economy: Industry (% of GVA)', 'Economy: Services and other activity (% of GVA)']]

In [9]:
country_data_clean.head()

,country,GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),Economy: Industry (% of GVA),Economy: Services and other activity (% of GVA)
0,Afghanistan,20270,-2.4,623.2,23.3,23.3,53.3
1,Albania,11541,2.6,3984.2,22.4,26.0,51.7
2,Algeria,164779,3.8,4154.1,12.2,37.3,50.5
3,American Samoa,-99,-99,-99.0,-99,-99.0,-99.0
4,Andorra,2812,0.8,39896.4,0.5,10.8,88.6


In [10]:
country_data_clean = country_data_clean.rename(columns={'country':'Country Name'})

In [11]:
country_data_clean.columns

Index(['Country Name', 'GDP: Gross domestic product (million current US$)',
       'GDP growth rate (annual %, const. 2005 prices)',
       'GDP per capita (current US$)', 'Economy: Agriculture (% of GVA)',
       'Economy: Industry (% of GVA)',
       'Economy: Services and other activity (% of GVA)'],
      dtype='object')

In [12]:
#Load in second dataset - Country heights.

country_height = pd.read_csv("country_height_data.csv")



In [13]:
country_height.columns

Index(['Rank', 'Country Name', 'Male Height in Cm', 'Female Height in Cm',
       'Male Height in Ft', 'Female Height in Ft'],
      dtype='object')

In [14]:
country_height = country_height[['Country Name','Male Height in Ft', 'Female Height in Ft']]

In [15]:
country_height.head()

,Country Name,Male Height in Ft,Female Height in Ft
0,Netherlands,6.03,5.59
1,Montenegro,6.01,5.58
2,Estonia,6.00,5.53
3,Bosnia and Herzegovina,5.99,5.49
4,Iceland,5.97,5.54


In [16]:
country_death_causes = pd.read_csv("country_death_causes.csv")

In [17]:
country_death_causes.columns

Index(['Country Name', 'Covid-19 Deaths', 'Cardiovascular diseases',
       'Respiratory diseases ', 'Kidney diseases', 'Neonatal disorders ',
       'Meningitis ', 'Malaria ', 'Interpersonal violence', 'HIV/AIDS',
       'Tuberculosis', 'Maternal disorders', 'Lower respiratory infections',
       'Alcohol use disorders', 'Diarrheal diseases', 'Poisoning',
       'Nutritional deficiencies', ' Alzheimer's disease',
       'Parkinson's disease', ' Acute hepatitis', 'Digestive diseases',
       ' Cirrhosis and other chronic liver diseases',
       'Protein-energy malnutrition', 'Neoplasms', 'Fire, heat', 'Drowning',
       'Drug use disorders', 'Road injuries',
       'Environmental heat and cold exposure', 'Self-harm',
       ' Conflict and terrorism', 'Diabetes ', 'Unnamed: 32'],
      dtype='object')

In [18]:
country_death_causes = country_death_causes[['Country Name', 'Covid-19 Deaths','Meningitis ', 'Malaria ','HIV/AIDS','Tuberculosis']]

In [19]:
country_death_causes.head()

,Country Name,Covid-19 Deaths,Meningitis,Malaria,HIV/AIDS,Tuberculosis
0,Afghanistan,2201.0,1563,530,318,3627
1,Albania,1181.0,13,0,2,11
2,Algeria,2762.0,292,0,264,445
3,Andorra,84.0,0,0,3,0
4,Angola,33.0,2520,10784,16802,11752


In [20]:
##Create Database "Countries"

cur, conn = create_database()

In [21]:
#Create first table - country data.
country_data_table_create = ("""CREATE TABLE IF NOT EXISTS countrydata(
Country_Name VARCHAR PRIMARY KEY,
GDP INT,
GDP_growth_rate INT,
GDP_per_capita INT,
Economy_Percent_Agriculture INT,
Economy_Percent_Industry INT,
Economy_Percent_Services INT)""")

In [22]:
cur.execute(country_data_table_create)
conn.commit()

In [31]:
#Create second table - country height data
country_height_table_create = ("""CREATE TABLE IF NOT EXISTS countrydata(
Country_Name VARCHAR,
Male_Height_in_Ft INT,
Female_Height_in_Ft INT )""")

cur.execute(country_height_table_create)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
